In [ ]:
!pip install mne

# Análisis inicial de señales EEG

In [ ]:
import mne
import matplotlib.pyplot as plt
import numpy as np

# Función para cargar un archivo EDF y realizar análisis inicial
def analyze_eeg(file_path):
    # Cargar el archivo EDF
    raw = mne.io.read_raw_edf(file_path, preload=True)

    # Mostrar información básica del archivo
    print(f"Información del archivo {file_path}:")
    print(raw.info)  # Información básica sobre el archivo, como canales, frecuencia de muestreo, etc.

    # Mostrar los primeros 5 canales de los datos
    print("\nCanales disponibles en el dataset:")
    print(raw.info['ch_names'])

    # Ver la frecuencia de muestreo
    print(f"\nFrecuencia de muestreo: {raw.info['sfreq']} Hz")

    # Visualización de los primeros 5 segundos de los datos
    raw.plot(duration=5, n_channels=5, scalings='auto')  # Graficar los primeros 5 segundos de las señales

    # Revisar si hay datos nulos (por si hay canales sin datos o con errores)
    print("\nRevisando datos nulos (si los hay)...")
    print(np.any(np.isnan(raw.get_data())))

    # Verificación de artefactos visualmente
    # Graficar 1 canal para revisar posibles artefactos
    raw.plot(duration=5, n_channels=1, scalings='auto')

    return raw

In [ ]:
import mne
import numpy as np

def check_zero_channels(edf_path, tol=1e-10):
    """
    Lee un archivo EDF y devuelve una lista de canales cuya señal es plana (≈0).

    Parámetros
    ----------
    edf_path : str
        Ruta al archivo .edf.
    tol : float
        Tolerancia para considerar que la desviación estándar es cero.

    Retorna
    -------
    zero_chs : list of str
        Nombres de canales con señal constante cero.
    """
    # Carga el EDF (sin mostrar logs de MNE)
    raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)

    data = raw.get_data()         # shape: (n_channels, n_times)
    zero_chs = []

    for idx, ch_name in enumerate(raw.ch_names):
        channel_data = data[idx]
        # revisa si la desviación estándar es menor que tol
        if np.std(channel_data) < tol:
            zero_chs.append(ch_name)

    return zero_chs

if __name__ == "__main__":
    edf_file = "/mnt/DGX0Raid/databases/CHB_data/CHB/chb01_03.edf"
    zeros = check_zero_channels(edf_file)
    if zeros:
        print(f"Canales con señal plana en {edf_file}: {zeros}")
    else:
        print(f"No se detectaron canales en cero en {edf_file}.")


<ipython-input-61-b63ac003d607>:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


No se detectaron canales en cero en /mnt/DGX0Raid/databases/CHB_data/CHB/chb01_03.edf.


In [ ]:
def list_channels_mne(edf_path):
    """
    Devuelve la lista de nombres de canales de un archivo EDF usando MNE.
    """
    raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
    return raw.ch_names

if __name__ == "__main__":
    edf_file = "/mnt/DGX0Raid/databases/CHB_data/CHB/chb01_03.edf"
    channels = list_channels_mne(edf_file)
    print(f"Canales en {edf_file}:")
    for ch in channels:
        print(f"  - {ch}")


Canales en /mnt/DGX0Raid/databases/CHB_data/CHB/chb01_03.edf:
  - FP1-F7
  - F7-T7
  - T7-P7
  - P7-O1
  - FP1-F3
  - F3-C3
  - C3-P3
  - P3-O1
  - FP2-F4
  - F4-C4
  - C4-P4
  - P4-O2
  - FP2-F8
  - F8-T8
  - T8-P8-0
  - P8-O2
  - FZ-CZ
  - CZ-PZ
  - P7-T7
  - T7-FT9
  - FT9-FT10
  - FT10-T8
  - T8-P8-1


<ipython-input-62-8d86af42a8c2>:5: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)


# Ruta DGX

In [ ]:
import os

# Cambia esta ruta por la que tú necesites
ruta = '/mnt/DGX0Raid/databases/CHB_data/CHB'
os.chdir(ruta)

# Verifica que estás en el directorio correcto
print("Directorio actual:", os.getcwd())


Directorio actual: /mnt/DGX0Raid/databases/CHB_data/CHB


In [ ]:
import glob
import re

def extraer_duraciones_archivo(ruta_archivo):
    """Extrae las duraciones de seizure de un archivo."""
    duraciones = []
    with open(ruta_archivo, "r") as archivo:
        lineas = archivo.readlines()
        # Recorremos cada línea para encontrar el inicio y fin del seizure
        for i, linea in enumerate(lineas):
            if "Start Time:" in linea and "Seizure" in linea:
                inicio = re.search(r"Start Time:\s*(\d+)", linea)
                # Buscamos en la siguiente línea el tiempo de finalización
                if inicio and i + 1 < len(lineas) and "End Time:" in lineas[i + 1] and "Seizure" in lineas[i + 1]:
                    fin = re.search(r"End Time:\s*(\d+)", lineas[i + 1])
                    if fin:
                        inicio_t = int(inicio.group(1))
                        fin_t = int(fin.group(1))
                        duracion = fin_t - inicio_t
                        # Imprimir los valores de inicio, fin y duración calculada
                        print(f"Start Time: {inicio_t} - End Time: {fin_t} -> Duración: {duracion} segundos")
                        duraciones.append(duracion)
    return duraciones

def obtener_maxima_duracion(patron_archivos):
    """
    Busca en todos los archivos que cumplan el patrón, imprime las duraciones de cada seizure episode,
    el número de seizures por archivo y el total de duraciones encontradas, devolviendo la mayor duración.
    """
    max_duracion = 0
    total_duraciones = 0
    # Obtenemos la lista de archivos que coincidan con el patrón
    lista_archivos = glob.glob(patron_archivos)

    for archivo in lista_archivos:
        print(f"\nProcesando archivo: {archivo}")
        duraciones = extraer_duraciones_archivo(archivo)
        total_duraciones += len(duraciones)
        if duraciones:
            print("Número de seizures en el archivo", archivo, ":", len(duraciones))
            max_archivo = max(duraciones)
            if max_archivo > max_duracion:
                max_duracion = max_archivo

    print("\nTotal de archivos procesados:", len(lista_archivos))
    print("Número total de duraciones encontradas:", total_duraciones)
    return max_duracion

# Uso del código:
if __name__ == "__main__":
    # Asumiendo que los archivos tienen extensión .txt y se encuentran en el directorio especificado
    patron = "/mnt/DGX0Raid/databases/CHB_data/txt_Files/*.txt"
    duracion_max = obtener_maxima_duracion(patron)
    print("\nLa mayor duración de un seizure episode es:", duracion_max, "segundos")


Procesando archivo: /mnt/DGX0Raid/databases/CHB_data/txt_Files/chb14-summary.txt
Start Time: 1986 - End Time: 2000 -> Duración: 14 segundos
Start Time: 1372 - End Time: 1392 -> Duración: 20 segundos
Start Time: 2817 - End Time: 2839 -> Duración: 22 segundos
Start Time: 1911 - End Time: 1925 -> Duración: 14 segundos
Start Time: 1838 - End Time: 1879 -> Duración: 41 segundos
Start Time: 3239 - End Time: 3259 -> Duración: 20 segundos
Start Time: 1039 - End Time: 1061 -> Duración: 22 segundos
Start Time: 2833 - End Time: 2849 -> Duración: 16 segundos
Número de seizures en el archivo /mnt/DGX0Raid/databases/CHB_data/txt_Files/chb14-summary.txt : 8

Procesando archivo: /mnt/DGX0Raid/databases/CHB_data/txt_Files/chb09-summary.txt
Start Time: 12231 - End Time: 12295 -> Duración: 64 segundos
Start Time: 2951 - End Time: 3030 -> Duración: 79 segundos
Start Time: 9196 - End Time: 9267 -> Duración: 71 segundos
Start Time: 5299 - End Time: 5361 -> Duración: 62 segundos
Número de seizures en el arc

# Diccionario con el nombre del archivo edf y el inicio y fin de los episodios

In [ ]:
import glob
import re

def extraer_episodios_desde_txt(ruta_archivo):
    """
    Lee un archivo de texto que contiene bloques con:
      File Name: chb01_01.edf
      File Start Time: ...
      File End Time: ...
      Number of Seizures in File: ...
      Seizure Start Time: ...
      Seizure End Time: ...
    y devuelve un diccionario donde cada clave es el nombre de archivo EDF (p.ej. 'chb01_01.edf'),
    y el valor es una lista de episodios con 'start' y 'end'.
    """
    dict_episodios = {}
    current_file_name = None  # Para ir guardando el nombre EDF actual

    with open(ruta_archivo, "r") as f:
        lineas = f.readlines()

        for i, linea in enumerate(lineas):
            # 1) Detectar si la línea contiene "File Name:"
            if "File Name:" in linea:
                # Extraer el nombre EDF, por ejemplo 'chb01_01.edf'
                match_file = re.search(r"File Name:\s*(\S+)", linea)
                if match_file:
                    current_file_name = match_file.group(1)
                    # Crear entrada vacía en el diccionario si no existe
                    if current_file_name not in dict_episodios:
                        dict_episodios[current_file_name] = []

            # 2) Detectar un Seizure Start Time y Seizure End Time
            if "Start Time:" in linea and "Seizure" in linea:
                start_match = re.search(r"Start Time:\s*(\d+)", linea)
                # Revisar la siguiente línea para el "Seizure End Time:"
                if start_match and (i + 1) < len(lineas) and "End Time:" in lineas[i+1] and "Seizure" in lineas[i+1]:
                    end_match = re.search(r"End Time:\s*(\d+)", lineas[i+1])
                    if end_match and current_file_name:
                        start_time = int(start_match.group(1))
                        end_time = int(end_match.group(1))
                        # Añadir este episodio al nombre de archivo EDF actual
                        dict_episodios[current_file_name].append({
                            "start": start_time,
                            "end": end_time
                        })

    return dict_episodios

def extraer_episodios_varios_txt(patron_archivos):
    """
    Recorre todos los archivos .txt que cumplan con el patrón (por ejemplo, "/ruta/*.txt"),
    y retorna un diccionario global donde cada clave es el nombre EDF (ej: 'chb01_01.edf')
    y su valor es la lista de episodios con 'start' y 'end' combinados.
    """
    dict_global = {}

    # Buscar todos los archivos que coinciden con el patrón
    lista_txt = glob.glob(patron_archivos)

    for ruta_txt in lista_txt:
        # Procesar el archivo .txt y obtener un diccionario parcial
        dict_parcial = extraer_episodios_desde_txt(ruta_txt)

        # Combinar con el diccionario global
        for edf_name, episodios in dict_parcial.items():
            if edf_name not in dict_global:
                dict_global[edf_name] = episodios
            else:
                # Si ya existe esa clave, concatenamos los nuevos episodios
                dict_global[edf_name].extend(episodios)

    return dict_global

if __name__ == "__main__":
    patron = "/mnt/DGX0Raid/databases/CHB_data/txt_Files/*.txt"
    dict_episodios = extraer_episodios_varios_txt(patron)

    # Calcular el número total de archivos procesados
    lista_txt = glob.glob(patron)
    total_archivos = len(lista_txt)

    # Calcular el número total de episodios encontrados
    total_episodios = sum(len(episodios) for episodios in dict_episodios.values())

    print(f"Total de archivos procesados: {total_archivos}")
    print(f"Número total de episodios encontrados: {total_episodios}")

    # Imprimir resultados por archivo EDF
    for edf_name, episodios in dict_episodios.items():
        print(f"\nEDF Name: {edf_name}")
        if episodios:
            for i, epi in enumerate(episodios, 1):
                print(f"  Episodio {i}: Start = {epi['start']}, End = {epi['end']}")
        else:
            print("  No se encontraron episodios para este archivo EDF.")


Total de archivos procesados: 24
Número total de episodios encontrados: 198

EDF Name: chb14_01.edf
  No se encontraron episodios para este archivo EDF.

EDF Name: chb14_02.edf
  No se encontraron episodios para este archivo EDF.

EDF Name: chb14_03.edf
  Episodio 1: Start = 1986, End = 2000

EDF Name: chb14_04.edf
  Episodio 1: Start = 1372, End = 1392
  Episodio 2: Start = 2817, End = 2839

EDF Name: chb14_06.edf
  Episodio 1: Start = 1911, End = 1925

EDF Name: chb14_07.edf
  No se encontraron episodios para este archivo EDF.

EDF Name: chb14_11.edf
  Episodio 1: Start = 1838, End = 1879

EDF Name: chb14_12.edf
  No se encontraron episodios para este archivo EDF.

EDF Name: chb14_13.edf
  No se encontraron episodios para este archivo EDF.

EDF Name: chb14_14.edf
  No se encontraron episodios para este archivo EDF.

EDF Name: chb14_16.edf
  No se encontraron episodios para este archivo EDF.

EDF Name: chb14_17.edf
  Episodio 1: Start = 3239, End = 3259

EDF Name: chb14_18.edf
  Episo

# Creación de listas normal y epiléptica con los nombres de los archivos

In [ ]:
import glob
import re

def clasificar_archivos_desde_txt(ruta_archivo):
    """
    Lee un archivo de texto que contiene información en líneas como:
      File Name: chb01_01.edf
      Number of Seizures in File: 0
    y clasifica el archivo como normal o con epilepsia según el número de seizures.

    Retorna dos listas:
      - normal_files (contiene los nombres de archivo EDF con 0 seizures)
      - epileptic_files (contiene los nombres de archivo EDF con >=1 seizures)
    """
    normal_files = []
    epileptic_files = []

    current_file_name = None

    with open(ruta_archivo, "r") as f:
        lineas = f.readlines()

        for linea in lineas:
            # Detectar el nombre del archivo EDF
            if "File Name:" in linea:
                match_file = re.search(r"File Name:\s*(\S+)", linea)
                if match_file:
                    current_file_name = match_file.group(1)

            # Detectar el número de seizures en el archivo
            if "Number of Seizures in File:" in linea:
                match_seizures = re.search(r"Number of Seizures in File:\s*(\d+)", linea)
                if match_seizures and current_file_name:
                    num_seizures = int(match_seizures.group(1))

                    if num_seizures == 0:
                        normal_files.append(current_file_name)
                    else:
                        epileptic_files.append(current_file_name)

                    # Reseteamos el current_file_name para el siguiente bloque
                    current_file_name = None

    return normal_files, epileptic_files

def clasificar_archivos_varios_txt(patron_archivos):
    """
    Recorre todos los archivos .txt que cumplan con el patrón y
    genera dos listas globales:
      - normal_files (archivos EDF con 0 seizures)
      - epileptic_files (archivos EDF con >=1 seizures)
    """
    normal_files = set()
    epileptic_files = set()

    lista_txt = glob.glob(patron_archivos)
    for ruta_txt in lista_txt:
        nf, ef = clasificar_archivos_desde_txt(ruta_txt)
        normal_files.update(nf)
        epileptic_files.update(ef)

    return list(normal_files), list(epileptic_files)

def ordenar_por_numero_chb(lista_archivos):
    """
    Ordena una lista de nombres de archivos según el número que aparece
    justo después de "chb". Por ejemplo, 'chb01_01.edf' se ordenará antes de 'chb10_03.edf'.
    """
    # Usamos re.IGNORECASE por si acaso el prefijo está en mayúsculas o minúsculas.
    return sorted(lista_archivos, key=lambda x: int(re.search(r'chb(\d+)', x, re.IGNORECASE).group(1)))

if __name__ == "__main__":
    patron = "/mnt/DGX0Raid/databases/CHB_data/txt_Files/*.txt"
    normal_list, epileptic_list = clasificar_archivos_varios_txt(patron)

    # Ordenar las listas según el número que sigue a "chb"
    normal_list_ordenada = ordenar_por_numero_chb(normal_list)
    epileptic_list_ordenada = ordenar_por_numero_chb(epileptic_list)

    print("Archivos normales (0 seizures) ordenados:")
    for f in normal_list_ordenada:
        print("  ", f)

    print("\nArchivos con epilepsia (>=1 seizures) ordenados:")
    for f in epileptic_list_ordenada:
        print("  ", f)

    # Si necesitas conocer el total de file names (únicos) en cada categoría:
    print("\nTotal de archivos normales:", len(normal_list_ordenada))
    print("Total de archivos con epilepsia:", len(epileptic_list_ordenada))


Archivos normales (0 seizures) ordenados:
   chb01_20.edf
   chb01_17.edf
   chb01_02.edf
   chb01_41.edf
   chb01_25.edf
   chb01_39.edf
   chb01_24.edf
   chb01_43.edf
   chb01_09.edf
   chb01_33.edf
   chb01_01.edf
   chb01_32.edf
   chb01_08.edf
   chb01_07.edf
   chb01_31.edf
   chb01_12.edf
   chb01_34.edf
   chb01_38.edf
   chb01_23.edf
   chb01_29.edf
   chb01_37.edf
   chb01_46.edf
   chb01_11.edf
   chb01_19.edf
   chb01_05.edf
   chb01_40.edf
   chb01_36.edf
   chb01_30.edf
   chb01_10.edf
   chb01_13.edf
   chb01_06.edf
   chb01_27.edf
   chb01_22.edf
   chb01_14.edf
   chb01_42.edf
   chb02_17.edf
   chb02_24.edf
   chb02_08.edf
   chb02_21.edf
   chb02_28.edf
   chb02_03.edf
   chb02_35.edf
   chb02_18.edf
   chb02_26.edf
   chb02_06.edf
   chb02_20.edf
   chb02_09.edf
   chb02_22.edf
   chb02_05.edf
   chb02_04.edf
   chb02_07.edf
   chb02_33.edf
   chb02_15.edf
   chb02_10.edf
   chb02_11.edf
   chb02_23.edf
   chb02_31.edf
   chb02_34.edf
   chb02_01.edf
   chb02_02.ed

# Muestreo de archivos edf normales y balanceo de clases

In [ ]:
import random

# Ejemplo: Supongamos que estos son tus datos.
# La lista 'epileptic_records' contiene 198 episodios epilepticos.
# La lista 'normal_records' contiene más de 400 registros normales.
# (En este ejemplo, usaremos números para ilustrar. En tu caso, serán tus objetos o registros.)

# Realizamos un muestreo aleatorio sin reemplazo para seleccionar 198 registros normales.çr
random.seed(42)  # Para reproducibilidad
balanced_normal_records = random.sample(normal_list_ordenada, 198)

# Ahora tenemos 198 registros epilepticos y 198 registros normales.
print("Número de registros epilepticos:", len(epileptic_list_ordenada))
print("Número de registros normales muestreados:", len(balanced_normal_records))

# Opcional: combinar ambas clases en un solo conjunto balanceado
balanced_dataset = {
    "epileptic": epileptic_list_ordenada,
    "normal": balanced_normal_records
}

# Imprimir para verificar
print("\nDatos balanceados:")
print("Episodios epilepticos:", balanced_dataset["epileptic"])
print("Registros normales muestreados:", balanced_dataset["normal"])


Número de registros epilepticos: 141
Número de registros normales muestreados: 198

Datos balanceados:
Episodios epilepticos: ['chb01_18.edf', 'chb01_04.edf', 'chb01_03.edf', 'chb01_16.edf', 'chb01_26.edf', 'chb01_15.edf', 'chb01_21.edf', 'chb02_19.edf', 'chb02_16+.edf', 'chb02_16.edf', 'chb03_34.edf', 'chb03_01.edf', 'chb03_36.edf', 'chb03_03.edf', 'chb03_35.edf', 'chb03_02.edf', 'chb03_04.edf', 'chb04_28.edf', 'chb04_05.edf', 'chb04_08.edf', 'chb05_16.edf', 'chb05_17.edf', 'chb05_06.edf', 'chb05_13.edf', 'chb05_22.edf', 'chb06_10.edf', 'chb06_13.edf', 'chb06_24.edf', 'chb06_04.edf', 'chb06_18.edf', 'chb06_09.edf', 'chb06_01.edf', 'chb07_19.edf', 'chb07_12.edf', 'chb07_13.edf', 'chb08_02.edf', 'chb08_05.edf', 'chb08_21.edf', 'chb08_11.edf', 'chb08_13.edf', 'chb09_06.edf', 'chb09_08.edf', 'chb09_19.edf', 'chb10_27.edf', 'chb10_20.edf', 'chb10_38.edf', 'chb10_89.edf', 'chb10_30.edf', 'chb10_31.edf', 'chb10_12.edf', 'chb11_82.edf', 'chb11_92.edf', 'chb11_99.edf', 'chb12_27.edf', 'chb12_1

In [ ]:
# Combina las listas de archivos normales y epilepticos
archivos_validos = set(balanced_normal_records + epileptic_list_ordenada)

# Crear nuevo diccionario filtrado
dict_filtrado = {k: v for k, v in dict_episodios.items() if k in archivos_validos}

# Verificación opcional
print(f"\nNúmero de archivos en el diccionario original: {len(dict_episodios)}")
print(f"Número de archivos en el nuevo diccionario filtrado: {len(dict_filtrado)}")

# Imprimir algunos resultados
for edf_name, episodios in list(dict_filtrado.items())[:5]:  # Solo muestra los primeros 5
    print(f"\nArchivo: {edf_name}")
    for i, epi in enumerate(episodios, 1):
        print(f"  Episodio {i}: Start = {epi['start']}, End = {epi['end']}")



Número de archivos en el diccionario original: 676
Número de archivos en el nuevo diccionario filtrado: 339

Archivo: chb14_01.edf

Archivo: chb14_02.edf

Archivo: chb14_03.edf
  Episodio 1: Start = 1986, End = 2000

Archivo: chb14_04.edf
  Episodio 1: Start = 1372, End = 1392
  Episodio 2: Start = 2817, End = 2839

Archivo: chb14_06.edf
  Episodio 1: Start = 1911, End = 1925


# Uso de GPU

In [ ]:
import torch

# Seleccionar el dispositivo: 'cuda' si hay GPU, de lo contrario 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

if device.type == 'cuda':
    print("Nombre de la GPU:", torch.cuda.get_device_name(0))

Usando dispositivo: cuda
Nombre de la GPU: Tesla V100-DGXS-32GB


# Verificación de canales con desviación estándar 0

In [ ]:
import mne
import numpy as np
import os
import matplotlib.pyplot as plt

# ------------------------------------------------------------
def resolve_duplicate_channels(raw):
    """Renombra duplicados usando sufijos para cada canal repetido."""
    ch_names = raw.info['ch_names']
    counts = {}
    mapping = {}
    for ch in ch_names:
        counts[ch] = counts.get(ch, 0) + 1
        if counts[ch] == 1:
            mapping[ch] = ch
        else:
            mapping[ch] = f"{ch}.{counts[ch]-1}"
    raw.rename_channels(mapping)
    return raw

def load_edf_lazy(edf_path):
    """Carga EDF sin precargar muestras en memoria."""
    return mne.io.read_raw_edf(edf_path, preload=False, verbose=False)

def get_segment(raw, start_sec, window_samples, sfreq, n_channels):
    """Extrae siempre window_samples a partir de start_sec (21 s)."""
    start_idx = int(start_sec * sfreq)
    end_idx   = start_idx + window_samples
    # recortamos al tamaño real
    data = raw.get_data(start=start_idx, stop=end_idx)
    # si faltan muestras, rellenamos con ceros
    if data.shape[1] < window_samples:
        pad = np.zeros((n_channels, window_samples))
        pad[:, :data.shape[1]] = data
        data = pad
    return data

# ------------------------------------------------------------
def process_edf_dictionary(dict_episodios, edf_folder, tol=1e-10):
    """
    Para cada archivo en dict_episodios:
      - Carga el EDF
      - Resuelve duplicados de nombre
      - Detecta y descarta canales planos (std < tol)
      - Imprime cuántos canales quedan
      - Extrae ventanas de 21 s etiquetadas
    """
    results = {}
    for file_name, episodes in dict_episodios.items():
        edf_path = os.path.join(edf_folder, file_name)
        raw = load_edf_lazy(edf_path)
        raw = resolve_duplicate_channels(raw)

        # 1) Detectar canales planos
        all_data = raw.get_data()              # (n_ch, n_times)
        zero_chs = [raw.ch_names[i]
                    for i in range(len(raw.ch_names))
                    if np.std(all_data[i]) < tol]

        if zero_chs:
            print(f"{file_name}: descartando {len(zero_chs)} canales planos → {zero_chs}")
            raw.drop_channels(zero_chs)
        else:
            print(f"{file_name}: no se detectaron canales planos.")

        # 2) Mostrar canales que quedan
        n_channels = len(raw.ch_names)
        print(f"{file_name}: canales para análisis = {n_channels}\n")

        # 3) Preparar extracción de segmentos
        sfreq = raw.info['sfreq']
        window_samples = int(21 * sfreq)

        # Extraemos datos por canal
        channel_segments = {ch: [] for ch in raw.ch_names}

        if not episodes:
            # sin eventos → primeros 21 s
            seg = get_segment(raw, 0, window_samples, sfreq, n_channels)
            for i, ch in enumerate(raw.ch_names):
                channel_segments[ch].append((seg[i], 0))
        else:
            # con eventos
            for epi in episodes:
                seg = get_segment(raw, epi['start'], window_samples, sfreq, n_channels)
                label = 1  # evento epiléptico
                for i, ch in enumerate(raw.ch_names):
                    channel_segments[ch].append((seg[i], label))

        results[file_name] = channel_segments

    return results

# ------------------------------------------------------------
if __name__ == "__main__":
    edf_folder    = "/mnt/DGX0Raid/databases/CHB_data/CHB"
    # dict_filtrado debe tener la forma {'chb14_39.edf': [...], ...}
    dict_ventanas = process_edf_dictionary(dict_filtrado, edf_folder)


<ipython-input-70-6b83f09a19a2>:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  return mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
<ipython-input-70-6b83f09a19a2>:23: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  return mne.io.read_raw_edf(edf_path, preload=False, verbose=False)


chb14_01.edf: descartando 5 canales planos → ['--0', '--1', '--2', '--3', '--4']
chb14_01.edf: canales para análisis = 23



<ipython-input-70-6b83f09a19a2>:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  return mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
<ipython-input-70-6b83f09a19a2>:23: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  return mne.io.read_raw_edf(edf_path, preload=False, verbose=False)


chb14_02.edf: descartando 5 canales planos → ['--0', '--1', '--2', '--3', '--4']
chb14_02.edf: canales para análisis = 23



<ipython-input-70-6b83f09a19a2>:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  return mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
<ipython-input-70-6b83f09a19a2>:23: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  return mne.io.read_raw_edf(edf_path, preload=False, verbose=False)


...

chb24_15.edf: no se detectaron canales planos.
chb24_15.edf: canales para análisis = 23



<ipython-input-70-6b83f09a19a2>:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  return mne.io.read_raw_edf(edf_path, preload=False, verbose=False)


chb24_17.edf: no se detectaron canales planos.
chb24_17.edf: canales para análisis = 23



<ipython-input-70-6b83f09a19a2>:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  return mne.io.read_raw_edf(edf_path, preload=False, verbose=False)


chb24_21.edf: no se detectaron canales planos.
chb24_21.edf: canales para análisis = 23



# Prueba con dos modelos

In [ ]:
import numpy as np
import pandas as pd
from tslearn.clustering import TimeSeriesKMeans
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    adjusted_rand_score, silhouette_score, classification_report
)
from concurrent.futures import ProcessPoolExecutor, as_completed
import matplotlib.pyplot as plt
import os

# --- 1. Cargar CSV ---
def load_csv_data(csv_path):
    df = pd.read_csv(csv_path).dropna()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str)
    subject_ids = df.iloc[:, 0].tolist()
    labels = df.iloc[:, 1].astype(int).to_numpy()
    signals = df.iloc[:, 2:].to_numpy()
    groups = [sid.split('_')[0] for sid in subject_ids]
    return signals, labels, groups

# --- 2. Métricas de evaluación ---
def evaluate_fold(X_train_all, y_train_all, train_idx, val_idx, n_init, tol, sakoe_chiba_radius):
    X_tr, y_tr = X_train_all[train_idx], y_train_all[train_idx]
    X_val, y_val = X_train_all[val_idx], y_train_all[val_idx]

    model = TimeSeriesKMeans(
        n_clusters=2,
        metric="dtw",
        metric_params={"sakoe_chiba_radius": sakoe_chiba_radius},
        n_init=n_init,
        tol=tol,
        random_state=42
    )
    model.fit(X_tr)
    y_pred = model.predict(X_val)

    if accuracy_score(y_val, y_pred) < accuracy_score(y_val, 1 - y_pred):
        y_pred = 1 - y_pred

    try:
        sil = silhouette_score(X_val, y_pred)
    except:
        sil = np.nan

    return {
        'accuracy': accuracy_score(y_val, y_pred),
        'precision': precision_score(y_val, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_val, y_pred, average='weighted', zero_division=0),
        'f1': f1_score(y_val, y_pred, average='weighted', zero_division=0),
        'ari': adjusted_rand_score(y_val, y_pred),
        'silhouette': sil
    }

# --- 3. Grid Search Paralelizado ---
def run_grid_search(X_train_all, y_train_all, groups_train, param_grid, sakoe_chiba_radius):
    logo = LeaveOneGroupOut()
    splits = list(logo.split(X_train_all, y_train_all, groups=groups_train))
    search_summary = []
    best_f1 = -np.inf
    best_params = {}

    for n_init, tol in param_grid:
        fold_results = []

        with ProcessPoolExecutor() as executor:
            futures = [
                executor.submit(evaluate_fold, X_train_all, y_train_all, train_idx, val_idx, n_init, tol, sakoe_chiba_radius)
                for train_idx, val_idx in splits
            ]
            for future in as_completed(futures):
                fold_results.append(future.result())

        avg = {k: np.nanmean([fr[k] for fr in fold_results]) for k in fold_results[0]}
        search_summary.append({'Model': 'TSKMeans', 'n_init': n_init, 'tol': tol, **avg})
        print(f"TSKMeans n_init={n_init}, tol={tol} → F1={avg['f1']:.4f}, Silhouette={avg['silhouette']:.4f}")

        if avg['f1'] > best_f1:
            best_f1 = avg['f1']
            best_params = {'n_init': n_init, 'tol': tol}

    return search_summary, best_params

# --- 4. Entrenamiento Final y Evaluación ---
def train_and_evaluate_final_model(X_train_all, y_train_all, X_test, y_test, best_params, sakoe_chiba_radius):
    model = TimeSeriesKMeans(
        n_clusters=2,
        metric="dtw",
        metric_params={"sakoe_chiba_radius": sakoe_chiba_radius},
        n_init=best_params['n_init'],
        tol=best_params['tol'],
        random_state=42
    )
    model.fit(X_train_all)

    y_pred_train = model.predict(X_train_all)
    y_pred_test = model.predict(X_test)

    if accuracy_score(y_train_all, y_pred_train) < accuracy_score(y_train_all, 1 - y_pred_train):
        y_pred_train = 1 - y_pred_train
    if accuracy_score(y_test, y_pred_test) < accuracy_score(y_test, 1 - y_pred_test):
        y_pred_test = 1 - y_pred_test

    print("\nTrain Report:\n", classification_report(y_train_all, y_pred_train, zero_division=0))
    print("Test Report:\n", classification_report(y_test, y_pred_test, zero_division=0))

    # Visualización
    centroids = model.cluster_centers_
    plt.figure(figsize=(12, 5))
    for c in range(2):
        plt.subplot(1, 2, c + 1)
        for ts in X_test[y_pred_test == c]:
            plt.plot(ts.ravel(), alpha=0.3)
        plt.plot(centroids[c].ravel(), linewidth=2)
        plt.title(f"Cluster {c}")
    plt.tight_layout()
    plt.suptitle("TSKMeans - Clusters en Test", y=1.02)
    plt.show()

    return {
        'Train Acc': accuracy_score(y_train_all, y_pred_train),
        'Train F1': f1_score(y_train_all, y_pred_train, average='weighted', zero_division=0),
        'Train ARI': adjusted_rand_score(y_train_all, y_pred_train),
        'Test Acc': accuracy_score(y_test, y_pred_test),
        'Test F1': f1_score(y_test, y_pred_test, average='weighted', zero_division=0),
        'Test ARI': adjusted_rand_score(y_test, y_pred_test)
    }

# --- 5. Ejecutar todo ---
if __name__ == "__main__":
    train_csv = "/content/train_data.csv"
    test_csv = "/content/test_data.csv"

    X_train_all, y_train_all, groups_train = load_csv_data(train_csv)
    X_test, y_test, _ = load_csv_data(test_csv)

    # Grid reducido + Sakoe-Chiba radius
    param_grid = [(3, 1e-4), (3, 1e-5)]
    sakoe_chiba_radius = 10

    print("\n=== Grid Search con Sakoe-Chiba Band ===")
    search_summary, best_params = run_grid_search(X_train_all, y_train_all, groups_train, param_grid, sakoe_chiba_radius)

    print("\n=== Mejor Configuración ===")
    print(best_params)

    print("\n=== Resumen Grid Search ===")
    print(pd.DataFrame(search_summary).to_string(index=False))

    print("\n=== Evaluación Final ===")
    final_metrics = train_and_evaluate_final_model(X_train_all, y_train_all, X_test, y_test, best_params, sakoe_chiba_radius)

    # Reporte resumen
    final_df = pd.DataFrame([{**best_params, **final_metrics}])
    print("\n=== Resumen Final de Desempeño ===")
    print(final_df.to_string(index=False))



=== Grid Search con Sakoe-Chiba Band ===


In [ ]:
import numpy as np
import pandas as pd
from tslearn.clustering import KShape
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    adjusted_rand_score, classification_report
)
from sklearn.model_selection import LeaveOneGroupOut, GroupShuffleSplit
import matplotlib.pyplot as plt

# --- 1. Generador directo en NumPy ---
def ts_data_generator(dict_ventanas):
    for subject_id, channel_dict in dict_ventanas.items():
        for channel, windows in channel_dict.items():
            for window, label in windows:
                yield np.array(window), label, subject_id  # NumPy directamente

# --- 2. Split inicial de sujetos ---
subjects = np.array(list(dict_ventanas.keys()))
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X=subjects, groups=subjects))
train_subjects = subjects[train_idx].tolist()
test_subjects = subjects[test_idx].tolist()

# --- 3. Función para cargar datos ---
def load_data_for_subjects(subjects_list):
    X_list, y_list = [], []
    for window, label, subj in ts_data_generator(dict_ventanas):
        if subj in subjects_list:
            X_list.append(window)
            y_list.append(label)
    return np.stack(X_list, axis=0), np.array(y_list)

X_test, y_test = load_data_for_subjects(test_subjects)

# --- 4. Grid Search sin silhouette ---
logo = LeaveOneGroupOut()
param_grid = {'n_init': [1, 3, 5], 'tol': [1e-4, 1e-5, 1e-6]}
best_f1 = -np.inf
best_params = {}
search_summary = []

print("\n=== Grid Search para KShape ===")
for n_init in param_grid['n_init']:
    for tol in param_grid['tol']:
        metrics = {m: [] for m in ['accuracy','precision','recall','f1','ari']}
        for tr_idx, val_idx in logo.split(train_subjects, train_subjects, train_subjects):
            tr_subjs = [train_subjects[i] for i in tr_idx]
            val_subjs = [train_subjects[i] for i in val_idx]
            X_tr, y_tr = load_data_for_subjects(tr_subjs)
            X_val, y_val = load_data_for_subjects(val_subjs)

            model = KShape(n_clusters=2, n_init=n_init, tol=tol, random_state=42)
            model.fit(X_tr)
            y_pred = model.predict(X_val)

            if accuracy_score(y_val, y_pred) < accuracy_score(y_val, 1 - y_pred):
                y_pred = 1 - y_pred

            metrics['accuracy'].append(accuracy_score(y_val, y_pred))
            metrics['precision'].append(precision_score(y_val, y_pred, average='weighted', zero_division=0))
            metrics['recall'].append(recall_score(y_val, y_pred, average='weighted', zero_division=0))
            metrics['f1'].append(f1_score(y_val, y_pred, average='weighted', zero_division=0))
            metrics['ari'].append(adjusted_rand_score(y_val, y_pred))

        avg = {m: np.mean(v) for m, v in metrics.items()}
        search_summary.append({'Model': 'KShape','n_init': n_init,'tol': tol, **avg})
        print(f"KShape n_init={n_init}, tol={tol} -> F1={avg['f1']:.4f}")
        if avg['f1'] > best_f1:
            best_f1 = avg['f1']
            best_params = {'n_init': n_init, 'tol': tol}

print(f"\nMejores KShape: {best_params}, F1={best_f1:.4f}")
print("\n=== Resumen Grid Search ===")
print(pd.DataFrame(search_summary).to_string(index=False))

# --- 5. Evaluación final con mejores parámetros ---
print(f"\n=== Evaluación Final KShape ===")
X_train, y_train = load_data_for_subjects(train_subjects)
model = KShape(n_clusters=2, n_init=best_params['n_init'], tol=best_params['tol'], random_state=42)
model.fit(X_train)

y_pred_train = model.predict(X_train)
if accuracy_score(y_train, y_pred_train) < accuracy_score(y_train, 1 - y_pred_train):
    y_pred_train = 1 - y_pred_train

print("Train Report:\n", classification_report(y_train, y_pred_train, zero_division=0))

y_pred_test = model.predict(X_test)
if accuracy_score(y_test, y_pred_test) < accuracy_score(y_test, 1 - y_pred_test):
    y_pred_test = 1 - y_pred_test

print("Test Report:\n", classification_report(y_test, y_pred_test, zero_division=0))

# --- 6. Visualización ---
centroids = model.cluster_centers_
plt.figure(figsize=(12, 5))
for c in range(2):
    plt.subplot(1, 2, c + 1)
    for ts in X_test[y_pred_test == c]:
        plt.plot(ts.ravel(), alpha=0.3)
    plt.plot(centroids[c].ravel(), linewidth=2)
    plt.title(f"KShape Cluster {c}")
plt.tight_layout()
plt.suptitle("KShape - Clusters en Test con Centroides", y=1.02)
plt.show()



=== Grid Search para KShape ===


/usr/local/lib/python3.10/dist-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  warnings.warn(
/usr/local/lib/python3.10

KShape n_init=1, tol=0.0001 -> F1=0.8933


/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tslearn/utils/utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional ti